## Import library

In [1]:
%run bert
%run utils.ipynb

import numpy as np
import pandas as pd
import sklearn as sk
from sklearn import linear_model 
from sklearn.metrics import cohen_kappa_score,mean_absolute_error,mean_squared_error,accuracy_score,explained_variance_score
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import optuna

## Load Data

In [2]:
loader = DataLoader()
d = loader.GetData('L2Writing')
#d = loader.GetShuffled()
d.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


## Model Inference

In [3]:
EmbeddingFetcher = GetBERTEmbeddings(d['full_text'],'model/deberta-v3-large')
EmbeddingFetcher.inf(stop=1500,SeqLen = 512 )

tokenized


c:\Users\GIGA\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\deberta_v2\modeling_deberta_v2.py:745: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_scores = torch.bmm(query_layer, key_layer.transpose(-1, -2)) / torch.tensor(
c:\Users\GIGA\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\deberta_v2\modeling_deberta_v2.py:829: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  score += c2p_att / torch.tensor(scale, dtype=c2p_att.dtype)
c:\Users\GIGA\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\deberta_v2\modeling_deberta_v2.py:852: UserWarning: To copy construct from a tensor, it is recommended 

0/1500, run:0
10/1500, run:0
20/1500, run:0
30/1500, run:0
40/1500, run:0
50/1500, run:0
60/1500, run:0
70/1500, run:0
80/1500, run:0
90/1500, run:0
100/1500, run:0
110/1500, run:0
120/1500, run:0
130/1500, run:0
140/1500, run:0
150/1500, run:0
160/1500, run:0
170/1500, run:0
180/1500, run:0
190/1500, run:0
200/1500, run:0
210/1500, run:0
220/1500, run:0
230/1500, run:0
240/1500, run:0
250/1500, run:0
260/1500, run:0
270/1500, run:0
280/1500, run:0
290/1500, run:0
300/1500, run:0
310/1500, run:0
320/1500, run:0
330/1500, run:0
340/1500, run:0
350/1500, run:0
360/1500, run:0
370/1500, run:0
380/1500, run:0
390/1500, run:0
400/1500, run:0
410/1500, run:0
420/1500, run:0
430/1500, run:0
440/1500, run:0
450/1500, run:0
460/1500, run:0
470/1500, run:0
480/1500, run:0
490/1500, run:0
500/1500, run:0
510/1500, run:0
520/1500, run:0
530/1500, run:0
540/1500, run:0
550/1500, run:0
560/1500, run:0
570/1500, run:0
580/1500, run:0
590/1500, run:0
600/1500, run:0
610/1500, run:0
620/1500, run:0
630

### Clear cuda cache

In [4]:
del EmbeddingFetcher.model
torch.cuda.empty_cache()

## Extract Embeddings

In [5]:
x = EmbeddingFetcher.GetEmbeddings('CLS')

## Split Train and Test data

In [7]:
x = [np.array(i).reshape(1024) for i in x]
columns = ['cohesion', 'syntax', 'vocabulary','phraseology', 'grammar', 'conventions']
TrainX,TestX = x[:2800],x[2800:]

In [8]:
TrainY,TestY = [],[]
for i in columns:
    TrainY.append(d[i].iloc[:2800])
    TestY.append(d[i].iloc[2800:])

## Hyper parameter tuning

In [ ]:
def objective(trial):
    params = {"random_state":trial.suggest_categorical("random_state", [42]),           
        'learning_rate' : trial.suggest_float('learning_rate', 0.01, 1),  
        "max_depth" : trial.suggest_int("max_depth", 5, 15),
        "alpha" : trial.suggest_float('alpha',0.9,1),
    }
    error = []
    for i in range(6):
        model = xgb.XGBRegressor(**params)
        model.fit(TrainX,TrainY[i])
        PredY = model.predict(TestX)
        e = mean_squared_error(TestY[i],PredY,squared=False)
        error.append(e)
    mcrmse = sum(error)/6
    return mcrmse
study = optuna.create_study()
study.optimize(objective, n_trials=100) 

In [ ]:
from optuna.visualization.matplotlib import plot_optimization_history
plot_optimization_history(study)

## No tuning

In [10]:
def objective():
    params = {          
        'learning_rate' : 0.042,                   
    }
    error = []
    for i in range(6):
        model = xgb.XGBRegressor(**params)
        model.fit(TrainX,TrainY[i])
        PredY = model.predict(TestX)
        e = mean_squared_error(TestY[i],PredY,squared=False)
        error.append(e)
    mcrmse = sum(error)/6
    return mcrmse,error
objective()

(0.48930992011706814,
 [0.5236888878939432,
  0.4829607151914506,
  0.4378622568825183,
  0.4774016171738794,
  0.5231214299712789,
  0.49082461358933865])